Polynomial-sized Linear Programming For Worst-case Analysis of Time-Sensitive Networks
====================
The work is based on the result of [Trade-off between accuracy and tractability of Network Calculus in FIFO networks](https://doi.org/10.1016/j.peva.2021.102250). 

In [13]:
import pulp
import numpy as np

Linear Program Solver of TFA
------------------
- Try to implement the toy example of Fig 6. of the paper
- This simple setting is without shaper

## Variables
All indices start with 0, (some indexing may differ from that in the paper)
### Network Params
- Graph: Adjacency matrix
- Flow: list of list, ordered as data flow
- Arrival constraints $\gamma_{b,r}$: row=flow arrival curve, col=[burst ($b$), rate ($r$)]
- Service constraints $\beta_{R, T}$: row=server service curve, col=[rate ($R$), latency ($L$)]

In [14]:
# Global arrival/service parameters
BURST    = 1 # arrival burst
ARR_RATE = 1 # arrival rate
LATENCY  = 1 # service latency
SER_RATE = 4 # service rate
# Network
ADJ_MAT = np.array([[0, 1], [0, 0]])    # network topology, from server 0 -> server 1
FLOWS = [[0,1], [0], [1]]                # data flows
NUM_FLOW = len(FLOWS)
NUM_SERVER = ADJ_MAT.shape[0]

ARRIVALS = np.repeat(np.array([[BURST, ARR_RATE]]), NUM_FLOW, axis=0)
SERVICES = np.repeat(np.array([[SER_RATE, LATENCY]]), NUM_SERVER, axis=0)


In [15]:
def get_flows(server: int) -> list:
    '''
    Given a server j, find the indices of flow Fl(j) that passes server j
    the answer is returned in a list
    '''
    idx = 0
    involved_flows = []
    for fl in FLOWS:
        if server in fl:
            involved_flows.append(idx)
        idx += 1

    return involved_flows

def get_successor(server: int) -> list:
    '''
    Given a server j, find the indices of servers which are successors of j
    the answer is returned in a list
    '''
    succ = np.argwhere(ADJ_MAT[server])
    if len(succ) == 0:
        return []
    else:
        return list(np.argwhere(ADJ_MAT[server])[0])

def var_set_name(name: str, *indices) -> str:
    '''
    Format the variable name. For example,
    base name: 'x', indices are 1 and 2, then the name is set as 'x_1,2'
    '''
    name += '_'
    for idx in indices:
        name += str(idx) + ','

    return name[:-1]

def var_get_name(name: str) -> tuple:
    '''
    Obtain the base name and indices from the formated variable name. For example,
    "x_1,2" -> ('x', [1,2])
    '''
    base_name, indices = name.split('_')
    indices = indices.split(',')
    indices = [int(idx) for idx in indices]
    
    return base_name, indices

### Define the Linear Program
**Variables**
- $s_j$ and $t_j$ are the time where data goes into server $j$ and leaves respectively. Here we use `in_time` and `out_time`.
- $d_j$ is the delay experienced at server $j$. Here we use `delays`.
- $A_i^{(j)}s_j$ is the cumulative data volume for flow $i$ that arrives at server $j$. Here we use `arrivals`.
- $D_i^{(j)}t_j$ is the cumulative data volume for flow $i$ that departs from server $j$. Here we use `departures`.
- $x_i^{(j)}$ is the possible burst experienced for flow $i$ that arrives at server $j$. Here we use `bursts`.

**Objective**

The objective is to maximize total delay $\sum_j d_j$.

In [16]:
# setup the lp problem for the TFA Linear program
tfa_prog = pulp.LpProblem('tfa_prog', pulp.LpMaximize)
# variables
in_time    = [None]*NUM_SERVER
out_time   = [None]*NUM_SERVER
delays     = [None]*NUM_SERVER
arrivals   = [[None]*NUM_SERVER for _ in range(NUM_FLOW)]
departures = [[None]*NUM_SERVER for _ in range(NUM_FLOW)]
bursts     = [[None]*NUM_SERVER for _ in range(NUM_FLOW)]

# for all server j
for server in range(NUM_SERVER):
    # Time constraints
    s_var = var_set_name('s', server)
    t_var = var_set_name('t', server)
    in_time[server]  = pulp.LpVariable(s_var, 0)
    out_time[server] = pulp.LpVariable(t_var)
    
    tfa_prog += in_time[server] <= out_time[server]    # add constraint that s <= t

    # Arrival constraints
    for fl in get_flows(server):
        As_var = var_set_name('As', fl, server)
        x_var  = var_set_name('x' , fl, server)
        Dt_var = var_set_name('Dt', fl, server)
        arrivals[fl][server]   = pulp.LpVariable(As_var)
        bursts[fl][server]     = pulp.LpVariable(x_var)
        departures[fl][server] = pulp.LpVariable(Dt_var)

        tfa_prog += arrivals[fl][server] <= bursts[fl][server] + ARRIVALS[fl][1]*in_time[server]

    # Service constraints
    tfa_prog += pulp.lpSum([departures[fl][server] for fl in get_flows(server)]) >= SERVICES[server][0]*out_time[server] - SERVICES[server][0]*SERVICES[server][1]
    tfa_prog += pulp.lpSum([departures[fl][server] for fl in get_flows(server)]) >= 0

    # FIFO constraints
    for fl in get_flows(server):
        tfa_prog += arrivals[fl][server] == departures[fl][server]

    # delays
    d_var = var_set_name('d', server)
    delays[server] = pulp.LpVariable(d_var)

    tfa_prog += delays[server] <= out_time[server] - in_time[server]

# Constraints on burst variables (x)
# Burst propagation
for server in range(NUM_SERVER):
    for fl in get_flows(server):
        for succ in get_successor(server):
            tfa_prog += bursts[fl][succ] <= bursts[fl][server] + ARRIVALS[fl][1]*delays[server]

for flow_idx, flow in enumerate(FLOWS):
    tfa_prog += bursts[flow_idx][flow[0]] <= ARRIVALS[fl][0]

# Set objective function
tfa_prog += pulp.lpSum(delays)


In [37]:
status = tfa_prog.solve(pulp.PULP_CBC_CMD(msg=False))

In [38]:
print("Status:", pulp.LpStatus[status])
print("Delays:\n--------")
total_delay = 0
for idx, d in enumerate(delays):
    print("d"+str(idx)+":", pulp.value(d))
    total_delay += pulp.value(d)
print("total =", total_delay)


Status: Optimal
Delays:
--------
d0: 1.5
d1: 1.875
total = 3.375


### Iterative method
Implementation of Algorithm 1 - TFA

In [35]:
b = 0
b_ij = np.zeros((NUM_FLOW, NUM_SERVER))
for idx, fl in enumerate(FLOWS):
    b_ij[idx, fl[0]] = ARRIVALS[idx, 0]
d_j = [0]*NUM_SERVER

for s in range(NUM_SERVER):
    fls = get_flows(s)
    b = np.sum(b_ij[fls, s])
    d_j[s] = SERVICES[s, 1] + b/SERVICES[s, 0]
    for fl in fls:
        for succ in get_successor(s):
            b_ij[fl, succ] = b_ij[fl, s] + ARRIVALS[fl, 1]*d_j[s]

print("Optimal d =", d_j)
print("Sum =", sum(d_j))

Optimal d = [1.5, 1.875]
Sum = 3.375


### TFA++
If shapers are added, using linear program

In [33]:
PKT_LENGTH = 0
CAPACITY = 4
SHAPERS = np.repeat(np.array([[PKT_LENGTH, CAPACITY]]), NUM_SERVER, axis=0)

# TFA++ (with shaper)
tfa_prog_pp = tfa_prog.copy()

# Adding shaping constraint
for s in range(NUM_SERVER):
    for succ in get_successor(s):
        flows_prev = set(get_flows(s))
        flows_next = set(get_flows(succ))
        mutual_flows = list(flows_prev.intersection(flows_next))
        tfa_prog_pp += pulp.lpSum(arrivals[fl][succ] for fl in mutual_flows) <= SHAPERS[s, 0] + SHAPERS[s, 1]*in_time[succ]

# Solve
status = tfa_prog_pp.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/54/0g1ch2216h50ggt6hgkp5vjw0000gn/T/7859a422ef734de4a8601d3dde1eb666-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/54/0g1ch2216h50ggt6hgkp5vjw0000gn/T/7859a422ef734de4a8601d3dde1eb666-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 80 RHS
At line 103 BOUNDS
At line 120 ENDATA
Problem MODEL has 22 rows, 18 columns and 50 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 9 (-13) rows, 6 (-12) columns and 22 (-28) elements
0  Obj -1.025 Primal inf 6.1333303 (3) Dual inf 1.249998 (2)
0  Obj -1.025 Primal inf 6.1333303 (3) Dual inf 3.3383333e+10 (5) w.o. free dual inf (3)
8  Obj 2.9583333
Optimal - objective value 2.9583333
After Postsolve,

In [34]:
print("Status:", pulp.LpStatus[status])
print("Delays:\n--------")
total_delay = 0
for idx, d in enumerate(delays):
    print("d"+str(idx)+":", pulp.value(d))
    total_delay += pulp.value(d)
print("total =", total_delay)

Status: Optimal
Delays:
--------
d0: 1.5
d1: 1.4583333
total = 2.9583333
